In [ ]:
from collections import Counter
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.utils.data import TensorDataset
from sklearn.metrics import classification_report
from data_preparation.converter_from_ud_to_txt import UDConverter
from train.train_pipeline import train_eval_loop, predict_with_model

In [1]:
!git clone https://github.com/IlyaSk10/POS_Tagger_with_using_CNN/
%cd POS_Tagger_with_using_CNN
!unzip SYNTAGRUS_texts.zip
!head syntagrus_full.ud

UDConverter.convert_from_conllu("syntagrus_full.ud", "syntagrus_fixed.txt")
!head syntagrus_fixed.txt
text_data= "syntagrus_fixed.txt"

Cloning into 'POS_Tagger_with_using_CNN'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 43 (delta 16), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (43/43), done.
/content/POS_Tagger_with_using_CNN
Archive:  SYNTAGRUS_texts.zip
  inflating: syntagrus_full.ud       
1	Начальник	начальник	NOUN	Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
2	областного	областной	ADJ	Case=Gen|Degree=Pos|Gender=Neut|Number=Sing
3	управления	управление	NOUN	Animacy=Inan|Case=Gen|Gender=Neut|Number=Sing
4	связи	связь	NOUN	Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing
5	Семен	семен	NOUN	Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
6	Еремеевич	еремеевич	NOUN	Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
7	был	быть	VERB	Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act
8	человек	человек	NOUN	Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
9	простой	простой	ADJ	Case=Nom|Degree=Pos|Ge

In [ ]:
def load_from_file(file):
  with open(file,'r',encoding='utf-8') as f:
    read_file=f.read().split('\n')
  return read_file

In [ ]:
dataset=load_from_file(text_data)

In [ ]:
import string
dataset = list(filter(None, dataset))
text = ''.join([('' if iteration.split('\t')[2]=='PUNCT' else ' ')+iteration.split('\t')[0] for iteration in dataset[:200]]).strip()
print(text)

Начальник областного управления связи Семен Еремеевич был человек простой, приходил на работу всегда вовремя, здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом" Муха". В приемной его с утра ожидали посетители,- кое-кто с важными делами, а кое-кто и с такими, которые легко можно было решить в нижестоящих инстанциях, не затрудняя Семена Еремеевича. Однако стиль работы Семена Еремеевича заключался в том, чтобы принимать всех желающих и лично вникать в дело. Приемная была обставлена просто, но по-деловому. У двери стоял стол секретарши, на столе- пишущая машинка с широкой кареткой. В углу висел репродуктор и играло радио для развлечения ожидающих и еще для того, чтобы заглушать голос начальника, доносившийся из кабинета, так_как, бесспорно, среди посетителей могли находиться и случайные люди. Кабинет отличался скромностью, присущей Семену Еремеевичу. В глубине стоял широкий письменный стол с бронзовыми чернильницами и перед ним два кожаных кресла. Сп

In [ ]:
def clear_text (dataset):
  clean_dataset=[]
  for sentence in dataset:
    if sentence.split('\t')[0]!='.' and sentence.split('\t')[2]!='PUNCT':
      clean_dataset.append([sentence.split('\t')[0].lower(),sentence.split('\t')[2]])
    elif sentence.split('\t')[0]=='.':
      clean_dataset.append([sentence.split('\t')[0].lower(),sentence.split('\t')[2]])
    else:
      continue
  return clean_dataset

In [ ]:
clean_dataset=clear_text(dataset)

In [ ]:
print(clean_dataset[:20])

[['начальник', 'NOUN'], ['областного', 'ADJ'], ['управления', 'NOUN'], ['связи', 'NOUN'], ['семен', 'NOUN'], ['еремеевич', 'NOUN'], ['был', 'VERB'], ['человек', 'NOUN'], ['простой', 'ADJ'], ['приходил', 'VERB'], ['на', 'ADP'], ['работу', 'NOUN'], ['всегда', 'ADV'], ['вовремя', 'ADV'], ['здоровался', 'VERB'], ['с', 'ADP'], ['секретаршей', 'NOUN'], ['за', 'ADP'], ['руку', 'NOUN'], ['и', 'CONJ']]


In [ ]:
def concatenate_words(dataset):
  concat_words=[]
  concat_labels=[]
  sentence=[]
  labels=[]
  for word in dataset:
    if word[1]!='PUNCT':
      sentence.append(word[0])
      labels.append(word[1])
    else:
      concat_words.append([word for word in sentence])
      concat_labels.append([label for label in labels])
      sentence.clear()
      labels.clear()
  return concat_words,concat_labels

In [ ]:
text,labels_sen=concatenate_words(clean_dataset)

In [ ]:
clean_dataset=[line for line in clean_dataset if line!='']
MAX_LEN_TOKEN=max(len(line[0]) for line in clean_dataset)
NUMBER_UNIQUE_TOKEN=len(set(line[0] for line in clean_dataset if line[1]!='PUNCT'))
MAX_LEN_SENTENCE=max(len(sentence) for sentence in text)
print('Максимальная длина токена', MAX_LEN_TOKEN)
print('Количество уникальных токенов' , NUMBER_UNIQUE_TOKEN)
print('Максимальная длина предложения' , MAX_LEN_SENTENCE)

Максимальная длина токена 31
Количество уникальных токенов 104251
Максимальная длина предложения 216


In [ ]:
labels = ['<NOTAG>'] + sorted({token[1] for token in clean_dataset})
label2id = {label: i+1 for i, label in enumerate(labels)}
print('Метки частей речи' , label2id)

Метки частей речи {'<NOTAG>': 1, 'ADJ': 2, 'ADP': 3, 'ADV': 4, 'CONJ': 5, 'DET': 6, 'H': 7, 'INTJ': 8, 'NOUN': 9, 'NUM': 10, 'PART': 11, 'PRON': 12, 'PUNCT': 13, 'VERB': 14}


In [ ]:

def build_vocab(text,pad_symbol='<PAD>'):
  sentence=' '.join(line[0].lower() for line in text)
  mydict = dict((j, i+1) for i, j in enumerate(set(sentence)))
  mydict.update({pad_symbol:max(mydict.values())+1})
  return mydict

def counter(dataset,most_freq_symbols):
  if most_freq_symbols is None:
    most_freq_symbols=5
  sentence=' '.join([word[0] for word in dataset])
  return Counter(list(sentence)).most_common(most_freq_symbols)    


In [ ]:
print('Символы словаря',build_vocab(text=clean_dataset[:10]),'\n','Наиболее частотные символы',counter(clean_dataset[:10],most_freq_symbols=10))

Символы словаря {'й': 1, 'м': 2, 'н': 3, 'д': 4, 'я': 5, 'б': 6, 'т': 7, 'ч': 8, 'к': 9, 'л': 10, 'в': 11, 'а': 12, ' ': 13, 'з': 14, 'ь': 15, 'с': 16, 'х': 17, 'ы': 18, 'о': 19, 'е': 20, 'п': 21, 'у': 22, 'р': 23, 'г': 24, 'и': 25, '<PAD>': 26} 
 Наиболее частотные символы [(' ', 9), ('е', 9), ('о', 7), ('л', 6), ('и', 6), ('н', 5), ('а', 4), ('с', 4), ('р', 4), ('в', 4)]


In [ ]:
vocab=build_vocab(text=clean_dataset)

A=np.zeros(shape=(len(text[:40000]),MAX_LEN_SENTENCE,MAX_LEN_TOKEN))
B=np.zeros(shape=(len(text[:40000]),MAX_LEN_SENTENCE))
for i in range(len(text[:40000])):
  for j,word in enumerate(text[i]):
    conv=[number for letter in word for symbol,number in vocab.items() if letter==symbol]
    conv1=[number for symbol,number in label2id.items() if labels_sen[i][j]==symbol]
    np.put(A[i,j],[k for k in range(len(word))],conv)
    np.put(B[i],j,conv1)


In [ ]:
A[1][:5]

array([[29.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.],
       [69., 79., 83., 64.,  3.,  5., 63.,  2.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.],
       [64., 82., 63.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.],
       [56.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.],
       [73., 15., 79., 30.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.]])

In [ ]:
B[1][:5]

array([ 3.,  9., 12.,  3.,  9.])

In [ ]:
def add_pad(words2id,text,pad_symbol,max_len_token,add_boundary=True):
  mat_with_boundary=np.zeros(shape=(len(text),MAX_LEN_SENTENCE,MAX_LEN_TOKEN+1),dtype=int)
  for i in range(len(text)):
    for j in range(len(text[i])):
      if add_boundary==True:
        mat_with_boundary[i,j]=np.insert(words2id[i,j],0,pad_symbol)
      else:
        continue
  return words2id if (add_boundary==False) else mat_with_boundary

In [ ]:
A=add_pad(A,text[:40000],pad_symbol=0,max_len_token=MAX_LEN_TOKEN,add_boundary=True)

In [ ]:
A[1,:10]

array([[ 0, 29,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 69, 79, 83, 64,  3,  5, 63,  2,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 64, 82, 63,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 56,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 73, 15, 79, 30,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 63, 70, 83,  7, 30, 27, 83,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 69, 63, 56, 64, 15, 83, 15, 64, 27, 83,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0

In [ ]:
B[1]

array([ 3.,  9., 12.,  3.,  9., 14.,  9., 12.,  3.,  2.,  9.,  5., 12.,
       11.,  3.,  6.,  6.,  4.,  4., 14., 14.,  3.,  2.,  9., 11., 14.,
        9.,  9.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(A, B, test_size=0.33, random_state=42)

from keras.utils import to_categorical

y_train_cat=to_categorical(y_train, num_classes=len(label2id.values())+1)
y_test_cat=to_categorical(y_test, num_classes=len(label2id.values())+1)

Using TensorFlow backend.


In [ ]:

class StackedConv1d(nn.Module):
  def __init__(self,features_num,layers_n=1,kernel_size=3,conv_layer=nn.Conv1d,dropout=0.0):
    super().__init__()
    layers=[]
    for _ in range(layers_n):
      layers.append(nn.Sequential(conv_layer(features_num,features_num,kernel_size,padding=kernel_size//2),
                                  nn.Dropout(dropout),
                                  nn.LeakyReLU()))
      self.layers=nn.ModuleList(layers)

  # simple ResNet
  def forward(self,x):
    for layer in self.layers:
      x=x+layer(x)
    return x

In [ ]:
class SingleTokenPOSTagger(nn.Module):
    def __init__(self, vocab_size, labels_num, embedding_size=32, **kwargs):
        super().__init__()
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.backbone = StackedConv1d(embedding_size, **kwargs)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Linear(embedding_size, labels_num)
        self.labels_num = labels_num
    
    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        batch_size, max_sent_len, max_token_len = tokens.shape
        tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len)
        
        char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize
        char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen
        
        features = self.backbone(char_embeddings)
        
        global_features = self.global_pooling(features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize
        
        logits_flat = self.out(global_features)  # BatchSize*MaxSentenceLen x LabelsNum
        logits = logits_flat.view(batch_size, max_sent_len, self.labels_num)  # BatchSize x MaxSentenceLen x LabelsNum
        logits = logits.permute(0, 2, 1)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits

In [ ]:
single_token_model=SingleTokenPOSTagger(len(vocab),len(label2id),embedding_size=64,layers_n=3,kernel_size=3,dropout=0.3)
print("Количество параметров", sum(np.product(t.shape) for t in single_token_model.parameters()))

Количество параметров 43342


In [ ]:
X_train=torch.from_numpy(X_train).type(torch.LongTensor)
y_train=torch.from_numpy(y_train).type(torch.LongTensor)
X_test=torch.from_numpy(X_test).type(torch.LongTensor)
y_test=torch.from_numpy(y_test).type(torch.LongTensor)

train_dataset=TensorDataset(X_train,y_train)
test_dataset=TensorDataset(X_test,y_test)

In [ ]:
(best_val_loss,
 best_single_token_model) = train_eval_loop(single_token_model,
                                            train_dataset,
                                            test_dataset,
                                            F.cross_entropy,
                                            lr=5e-3,
                                            epoch_n=10,
                                            batch_size=64,
                                            device='cuda',
                                            early_stopping_patience=5,
                                            max_batches_per_epoch_train=500,
                                            max_batches_per_epoch_val=100,
                                            lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2,
                                                                                                                       factor=0.5,
                                                                                                                       verbose=True))

Эпоха 0
Эпоха: 501 итераций, 282.44 сек
Среднее значение функции потерь на обучении 0.08025922445197424
Среднее значение функции потерь на валидации 0.036914107103896615
Новая лучшая модель!

Эпоха 1
Эпоха: 501 итераций, 282.22 сек
Среднее значение функции потерь на обучении 0.028614820871583953
Среднее значение функции потерь на валидации 0.02822994330141804
Новая лучшая модель!

Эпоха 2
Эпоха: 501 итераций, 282.23 сек
Среднее значение функции потерь на обучении 0.02423277378320218
Среднее значение функции потерь на валидации 0.026719287074733488
Новая лучшая модель!

Эпоха 3
Эпоха: 501 итераций, 282.22 сек
Среднее значение функции потерь на обучении 0.022381869048073502
Среднее значение функции потерь на валидации 0.022604787897429254
Новая лучшая модель!

Эпоха 4
Эпоха: 501 итераций, 282.22 сек
Среднее значение функции потерь на обучении 0.02125430090014568
Среднее значение функции потерь на валидации 0.02318061840268645

Эпоха 5
Эпоха: 501 итераций, 282.22 сек
Среднее значение функ

In [ ]:
torch.save(best_single_token_model.state_dict(), '/model/single_token_pos.pth')

In [ ]:
single_token_model.load_state_dict(torch.load('/model/single_token_pos.pth'))

In [ ]:
test_pred = predict_with_model(single_token_model, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(y_test_cat))
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))

100%|██████████| 206/205.75 [00:03<00:00, 57.84it/s]


Среднее значение функции потерь на валидации 0.020207911729812622
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   1231232
         ADJ       0.84      0.94      0.89     11222
         ADP       1.00      0.99      0.99     10585
         ADV       0.84      0.88      0.86      6165
         AUX       0.87      0.63      0.73      1106
       CCONJ       0.89      0.98      0.93      4410
         DET       0.75      0.87      0.81      3085
        INTJ       0.00      0.00      0.00        11
        NOUN       0.98      0.91      0.94     27974
         NUM       0.96      0.87      0.91      1829
        PART       0.97      0.77      0.86      3877
        PRON       0.94      0.78      0.85      5598
       PROPN       0.77      0.95      0.85      4438
       PUNCT       1.00      1.00      1.00     22694
       SCONJ       0.77      0.74      0.75      2258
         SYM       1.00      0.96      0.98        53
        VERB   